# Plot GCP with lidar reflectance and IR images

In [ ]:
import os
from glob import glob
import pandas as pd
import rioxarray as rxr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Define input files
data_folder = '/Users/rdcrlrka/Research/Soo_locks'
dem_fn = os.path.join(data_folder, '20251001_TLS', '2025-10-01-soo-model_5mm_Ground_Z.tif')
refl_fn = os.path.join(data_folder, '20251001_TLS', '2025-10-01-soo-model_5mm_Ground_Reflectance.tif')
X_fn = os.path.join(data_folder, '20251001_TLS', '2025-10-01-soo-model_5mm_Ground_X.tif')
Y_fn = os.path.join(data_folder, '20251001_TLS', '2025-10-01-soo-model_5mm_Ground_Y.tif')
img_folder = os.path.join(data_folder, '20251001_imagery', 'frames_IR')
gcp_fn = os.path.join(data_folder, '20251001_imagery', 'GCP.xlsx')
out_folder = os.path.join(img_folder + '_proc_out', 'figures')
os.makedirs(out_folder, exist_ok=True)

# Load GCP and reflectance image
gcp = pd.read_excel(gcp_fn)
def load_raster(fn):
    raster = rxr.open_rasterio(fn).squeeze()
    raster = xr.where(raster==-9999, np.nan, raster)
    return raster
refl = load_raster(refl_fn)
X = load_raster(X_fn)
Y = load_raster(Y_fn)

In [ ]:
# Iterate over images in GCP
for img_name in tqdm(gcp['image_path'].drop_duplicates().values):

    print(img_name)

    # load image
    img_fn = os.path.join(img_folder, img_name)
    img = rxr.open_rasterio(img_fn)

    fig_fn = os.path.join(out_folder, f"gcp_{os.path.basename(img_fn)}.png")
    if os.path.exists(fig_fn):
        continue

    # get all the GCP for that image
    gcp_img = gcp.loc[gcp['image_path']==img_name].reset_index(drop=True)

    # plot
    fig, ax = plt.subplots(1, 2, figsize=(14,6))
    ax[0].imshow(img.data[0], cmap='Grays_r', 
                 extent=(min(img.x), max(img.x), min(img.y), max(img.y)))
    ax[1].imshow(refl.data, cmap='Grays_r',
                 extent=(min(refl.x), max(refl.x), min(refl.y), max(refl.y)))
    
    # plot GCP
    xmin, ymin, xmax, ymax = np.inf, np.inf, -np.inf, -np.inf
    for i, row in gcp_img.iterrows(): 
        # reflectance coords
        samp = xr.where((np.abs(X - row['X']) < 1e-2) & 
                        (np.abs(Y - row['Y']) < 1e-2), 1, 0)
        if len(samp.where(samp==1, drop=True).x.values) < 1:
            print(i, 'Skipping a GCP, no good coords found')
            continue
        x_samp = samp.where(samp==1, drop=True).x.values[0]
        y_samp = samp.where(samp==1, drop=True).y.values[0]
        ax[1].plot(x_samp, y_samp, 'o', linewidth=0.5, 
                   markerfacecolor=plt.cm.Set1(i/len(gcp_img)), markeredgecolor='w', markersize=5)
        # adjust limits
        if x_samp < xmin:
            xmin = x_samp
        if x_samp > xmax:
            xmax = x_samp
        if y_samp < ymin:
            ymin = y_samp
        if y_samp > ymax:
            ymax = y_samp

        # image coords
        x_mesh, y_mesh = np.meshgrid(img.x.data, img.y.data)
        x_samp_img = float(x_mesh[int(-row['sample_row']), int(row['sample_col'])])
        y_samp_img = float(y_mesh[int(-row['sample_row']), int(row['sample_col'])])
        ax[0].plot(x_samp_img, y_samp_img, 'o', linewidth=0.5,
                   markerfacecolor=plt.cm.Set1(i/len(gcp_img)), markeredgecolor='w', markersize=5)

    ax[0].set_title('Camera image')
    ax[1].set_title('TLS reflectance')
    for axis in ax:
        axis.set_xticks([])
        axis.set_yticks([])

    ax[1].set_xlim(xmin-0.5, xmax+0.5)
    ax[1].set_ylim(ymin-0.5, ymax+0.5)

    fig.suptitle(img_name)
    fig.tight_layout()

    plt.show()

    # save figure to file
    fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
    print('Figure saved to file:', fig_fn)

In [ ]:
# Save GCP to file as TXT for ASP
gcp_out_fn = gcp_fn.replace('.xlsx', '.gcp')
gcp.to_csv(gcp_out_fn, sep=' ', header=False, index=False)
print('GCP saved to file for ASP:', gcp_out_fn)